# Python Deep Dive Part 4 - Object Oriented Programming

## Introduction

### Course topics

* Focus on Object Oriented Programming (OOP) concepts in Python
  * classes (types) and objects
  * functions, methods, and properties
  * polymorphism and special dunder methods
  * single inheritance
  * descriptors
  * enumerations
  * exceptions
  * 